**Authors:** Anowar Shajib, David Law

*Adapted from the [notebook](https://github.com/JWST-Templates/Notebooks/blob/main/MIRI_MRS_reduction_SPT0418-47_PAH_ch3long.ipynb) by the TEMPLATES team (J. Spilker, K. A. Phadke, D. Law).*

In this notebook we process our data through the Spec2 pipeline in order to produce 
Lvl2b data products (i.e., calibrated slope images and quick-look data cubes and 1d spectra).  
    
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html

During stage 2 of the pipeline, the countrate (slope) image products from stage 1, which have units of DN/s, are converted to units of surface brightness (MJy/sr) for both extended and point sources (as of DMS build 9.3/CAL_VER 1.10.2). 

If Pixel-based background subtraction was chosen, this will be applied during Spec2. 
Doing the Pixel based background subtraction requires an association file to be created to associate the background files with individual science files.

In [ ]:
import numpy as np
from tqdm import tqdm
import pathlib
from scipy.ndimage import median_filter
from util import *

In [ ]:
run_pipeline = True
run_on_nsclean_data = True
do_extra_processing = True
subtract_pixel_based_background = False
# Pixel-based background subtraction in spec2 (direct pixel subtraction) -Deep background exposures needed to not add noise.

if run_on_nsclean_data:
    stage1_directory = stage1_nsclean_directory
    stage2_directory = stage2_nsclean_directory
else:
    stage1_directory = stage1_processed_directory
    stage2_directory = stage2_directory

In [ ]:
# Define a function that will call the spec2 pipeline with our desired set of parameters
# We'll list the individual steps just to make it clear what's running
def run_spec2_pipeline(file_name, output_directory, no_cubes=False,
                       is_background=False
                       ):
    """
    Run the Spec2Pipeline on a given file.

    :param file_name: str, name of fits file to run pipeline on
    :param output_directory: str, path to output directory
    :param no_cubes: bool, if True, skip cube building and 1d spectral extraction
    :return: None
    :outputs:
    """
    print("Running Detector2Pipeline on {0:s}...".format(file_name))

    # Set default configuration from CRDS param reference files.
    # -This is required when running the pipeline in a function.
    crds_config = Spec2Pipeline.get_config_from_reference(file_name)
    spec2 = Spec2Pipeline.from_config_section(crds_config)

    spec2.output_dir = output_directory

    # Assign_wcs overrides
    # spec2.assign_wcs.override_distortion = 'myfile.asdf'
    # spec2.assign_wcs.override_regions = 'myfile.asdf'
    # spec2.assign_wcs.override_specwcs = 'myfile.asdf'
    # spec2.assign_wcs.override_wavelengthrange = 'myfile.asdf'

    # Background overrides were set up above
    if subtract_pixel_based_background:
        spec2.bkg_subtract.skip = False
    else:
        spec2.bkg_subtract.skip = True

    # Flatfield overrides
    # spec2.flat_field.override_flat = 'myfile.fits'

    # Straylight overrides
    # spec2.straylight.override_mrsxartcorr = 'myfile.fits'

    # Fringe overrides
    # spec2.fringe.override_fringe = 'myfile.fits'

    # Photom overrides
    # spec2.photom.override_photom = 'myfile.fits'

    # Cubepar overrides
    # spec2.cube_build.override_cubepar = 'myfile.fits'

    # Extract1D overrides
    # spec2.extract_1d.override_extract1d = 'myfile.asdf'
    # spec2.extract_1d.override_apcorr = 'myfile.asdf'

    # Overrides for whether or not certain steps should be skipped
    # spec2.assign_wcs.skip = False
    # spec2.bkg_subtract.skip = True
    # spec2.flat_field.skip = False
    # spec2.srctype.skip = False
    # spec2.straylight.skip = False
    # spec2.fringe.skip = False
    # spec2.photom.skip = False
    # spec2.residual_fringe.skip = False #does a residual fringe correction across the entire cube
    # spec2.cube_build.skip = False
    # spec2.extract_1d.skip = False

    spec2.flat_field.override_sflat = "./jwst_nirspec_sflat_0194_cleaned.fits"

    # Run pixel replacement code to extrapolate values for otherwise bad pixels.
    # This can help mitigate small 5-10% negative dips in spectra of bright sources.
    spec2.pixel_replace.skip = True
    # spec2.pixel_replace.algorithm='mingrad'

    # This nocubes option allows us to skip the cube building and 1d spectral extraction for individual
    # science data frames, but run it for the background data (as the 1d spectra are needed later
    # for the master background step in Spec3)
    if no_cubes:
        spec2.cube_build.skip = True
        spec2.extract_1d.skip = True
    
    if is_background:
        spec2.extract_1d.skip = False
        spec2.bkg_subtract.skip = True
        
    # Some cube building options
    # spec2.cube_build.weighting='drizzle'
    # spec2.cube_build.coord_system='ifualign' # If aligning cubes with IFU axes instead of sky

    spec2.save_results = True
    spec2(file_name)

In [ ]:
rate_files = sorted(glob.glob(stage1_processed_directory + "/*nrs1_rate.fits"))

len(rate_files)

In [ ]:
# Step through each of the science files, using relevant associated backgrounds in spec2 processing
# The background files are used in this step to perform pixel-based background subtraction (if desired)
# Otherwise Background subtraction is done later with Spec3 files
if run_pipeline:
    for file in tqdm(rate_files):
        if "01794002001" in file:
            is_background = True
        else:
            is_background = False
        
        run_spec2_pipeline(file, output_directory=stage2_directory, no_cubes=True,
                           is_background=is_background)
else:
    print("Skipping Spec2 processing for SCI data")

# Extra processing to flag bad pixels again

In [ ]:
if do_extra_processing:
    files = np.array(sorted(glob.glob(stage2_directory + "/*nrs1_cal.fits")))

    for file in files:
        with fits.open(file) as hdu:
            sci = hdu["SCI"].data

            # Flag positive pixels
            temp = sci.copy()
            temp[np.isfinite(temp) != True] = 0.0
            sci2 = median_filter(temp, size=(1, 11))
            diff = temp - sci2 * 2
            index = np.where(diff > 100)
            sci[index] = np.nan
            # Grow by 1 pixel in both directions
            y_index = index[1]
            x_index = index[0]
            n_index = len(y_index)
            for i in range(n_index):
                sci[x_index[i] - 1, y_index[i]] = np.nan
                sci[x_index[i] + 1, y_index[i]] = np.nan
                sci[x_index[i], y_index[i] - 1] = np.nan
                sci[x_index[i], y_index[i] + 1] = np.nan

            # Flag negative pixels
            temp = sci.copy()
            temp[np.isfinite(temp) != True] = 0.0
            sci2 = median_filter(temp, size=(1, 11))
            diff = temp - sci2 * 0.5
            index = np.where(diff < -100)
            sci[index] = np.nan
            # Grow by 1 pixel in both directions
            y_index = index[1]
            x_index = index[0]
            n_index = len(y_index)
            for i in range(n_index):
                sci[x_index[i] - 1, y_index[i]] = np.nan
                sci[x_index[i] + 1, y_index[i]] = np.nan
                sci[x_index[i], y_index[i] - 1] = np.nan
                sci[x_index[i], y_index[i] + 1] = np.nan

            hdu["SCI"].data = sci
            name = pathlib.Path(file).name
            hdu.writeto(stage2_processed_directory + name, overwrite=True)